# **The code below is to correct for water level logger drift, check accuracy of logger deployment and add the deployment block information to the block table.**

It does not require any Python knowledge, but it does require Python and certain packages to be installed on your computer. Read the code comments embedded in each code block carefully (denoted by '#'), as some components require user input (initials, manual water level measurements, etc.).

So far, this is tailored to *Solinst brand* data logger files. We will need some extra components for other brands, but at  this point I don't know what those data files look like.


---



**Connect to Shared Drive**

You will be prompted to click on a link that will show you an authorization code. copy the authorization code into the input box below. You also may be asked to allow Google Colab access to your Drive.

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


**Import packages**

You may need to install these packages on your computer, if they're not already. To do this through a command prompt on Windows, Python needs to be discoverable by your PATH variable (add the directory location of Python OR Python packages within a virtual environment to your system user variables). You can then either install directly or use pip installer ("pip install *package name*") through the terminal.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib auto
import numpy as np
from datetime import date

Using matplotlib backend: agg


In [4]:
path='/content/drive/My Drive/CZN_hydro/Water/preprocess_files/'
#Change file name below
file='Eb41-18_200807_LTC.csv'
df=pd.read_csv(path+file)

start_date=df['Date'][0]
end_date=df['Date'][len(df['Date'])-1]
print('Need manual measurements collected on '+start_date+' and '+end_date)

Need manual measurements collected on 8/7/2020 and 11/10/2020


Lookup manual measurements and add as variables

In [5]:
start_lev=2.90
end_lev=2.31

#Combine date and time fields, round to nearest 15 min increment 00:15:00) and set to index
df['DateTime']=pd.to_datetime(df['Date']+" "+df['Time'])
df['DateTime']=pd.to_datetime(df['DateTime']).dt.round('15min').dt.strftime('%m/%d/%Y %H:%M:%S')
df=df.set_index('DateTime')
#Add extra DateTime field for interactive plot
df['DateTime']=pd.to_datetime(df['Date']+" "+df['Time'])

Plot Water Level data

In [ ]:
df.plot(x='DateTime',y='LEVEL',style='.',rot=45)
plt.grid()
plt.tight_layout()
plt.show()

**Below are some common corrections to remove noisy or rogue data points**

All require some manual entry (i.e. number of rows, date and time). Zoom in and pan through the interactive plot to find info for specific points or periods of data. You can run one or more separately.

In [ ]:
#To remove a specified number of records at the start of the file
#Enter the number of rows below as start_del
start_del=1
df=df.iloc[start_del:]

In [ ]:
#To remove a specified number of records at the end of the file
#Enter the number of rows below as end_del
end_del=1
df=df.iloc[:-end_del]

In [ ]:
#To interpolate values between two records (i.e. smooth over a point)
#Change the date and time to that of whatever point you want to smooth over
#Can do this multiple times for multiple points and the interpolate function will do all at once
point=(df.index.get_loc('08/28/2020 14:30:00'))
df['LEVEL'][point]=np.nan
df['LEVEL']=df['LEVEL'].interpolate(axis=0)
print(df['LEVEL'][point])

In [ ]:
#To interpolate values over multiple records (i.e. interpolate over multiple consecutive records)
#Change the date and time for the first (first_pt) and last (last_pt) of the interval
first_pt=df.index.get_loc('08/28/2020 14:30:00')
last_pt=df.index.get_loc('08/28/2020 15:30:00')+1
df['LEVEL'][first_pt:last_pt]=np.nan
df['LEVEL']=df['LEVEL'].interpolate(axis=0)

**After removing and cleaning all noisy data, run the code below to correct water level values for sensor drift**

Your starting level (first record value) should match your starting manual measurement and your ending level (last record value) should match your end manual measurement.

In [ ]:
df['LEVEL_corr']=df['LEVEL']
n=len(df['LEVEL'])-1
LTCinit=df['LEVEL'][0]
LTCend=df['LEVEL'][n]
acc=(end_lev-LTCend)-(start_lev-LTCinit)
K=acc/(n-1)

new_lc=[]
for index,val in enumerate(df['LEVEL'],start=1):
    corr=(start_lev-LTCinit)+K*(index-1)
    new=round(val+corr,2)
    new_lc.append(new)

df['LEVEL_corr']=new_lc

**Add record data to block table**

In [ ]:
#Open existing block table from GDrive
df_block=pd.read_csv(path+'block.csv')

#Get block table info
block_start_time=df.index[0]
block_end_time=df.index[len(df['Date'])-1]
site_id=file.split('_')[0]
blockno=df_block['BlockNo'].max()+1
ind1=df_block['Index2'].max()+1
ind2=ind1+n
process_date=date.today().strftime('%Y-%m-%d')

def find_drift():
    if dat_type=='W':
        return np.nan
    elif dat_type=='T':
        return np.nan
    elif dat_type=='C':
        return (end_cal-start_cal)
    else:
        print('You have entered an invalid data type')

Manually enter some variables below

In [ ]:
drift=find_drift()

#Enter the variables below
initials='RWM' 
dat_type='W'
device='SolinstM3001'
sensor_sn=89864567
unit='F'
datum='TOC'
interval=15


In [ ]:
#Append to block table
df_block=df_block.append({'BlockNo':blockno,
                          'Site_id':site_id,
                          'Start_time':block_start_time,
                          'Index1':ind1,
                          'End_time':block_end_time,
                          'Index2':ind2,
                          'Type':dat_type,
                          'Device':device,
                          'Sensor_sn':sensor_sn,
                          'Unit':unit,
                          'Interval':interval,
                          'Datum':datum,
                          'Accuracy':acc,
                          'Drift':drift,
                          'Process_initials':initials,
                          'Process_date':process_date},ignore_index=True)